In [1]:
import scipy
import scipy.io
import numpy as np
from scipy.linalg import norm
from scipy.sparse import csr_matrix
from scipy.linalg import norm
import pickle
import time
from collections import defaultdict
import json
from sklearn.datasets import load_svmlight_file
import matplotlib.pyplot as plt
import pandas as pd
import numpy.matlib

In [ ]:
# alpha policies
def alpha_standard(k):
    return 2 / (k + 2)

def alpha_icml(Gap, hess_mult_v,d,Mf,nu):
    e = hess_mult_v ** 0.5
    bet = norm(d)
    if nu==2:
        delta=Mf*bet
        t=1/delta*np.log(1+Gap/(delta*e**2))
    elif nu==3:
        delta=1/2*Mf*e
        t = Gap / (Gap*delta + e**2)
    else:
        delta=(nu-2)/2*Mf*(beta**(3-nu))*e**(nu-2)
        if nu==4:
            t=1/delta*(1-np.exp(-delta*Gap/(e**2)))
        elif nu<4 and nu>2:
            const=(4-nu)/(nu-2)
            t=1/delta*(1-(1+(-delta*Gap*const/(e**2)))**(-1/const))
    return min(1, t)


def alpha_line_search(k,grad_function,delta_x,beta,accuracy):
    lb=grad_function(0).T.dot(delta_x);
    t_lb=0
    ub=grad_function(beta).T.dot(delta_x);
    t_ub=beta
    t=t_ub
    while t_ub<1 and ub<0:
        t_ub=1-(1-t_ub)/2
        ub=grad_function(t_ub).T.dot(delta_x);
    while t_ub-t_lb>accuracy:
        t=(t_lb+t_ub)/2
        val=grad_function(t).T.dot(delta_x);
        if val>0:
            t_ub=t
        else:
            t_lb=t 
    return t


In [2]:
def frank_wolfe(fun_x,
                grad_x,
                grad_beta,
                hess_mult_x,
                extra_fun,
                Mf,
                nu,
                linear_oracle,
                x_0,
                FW_params,
                alpha_policy='standard',
                eps=0.001,
                print_every=100,
                debug_info=False):
    """
        fun_x -- function, outputs function value and extra parameters
        grad_x -- grad value
        grad_beta -- grad velue for convex combination
        hess_mult_x -- Hessian multiplies by x on both sides
        extra_fun -- extra function used to transfer to other methods
        Mf --parameter for self concordence
        x_0 -- starting point (n)
        alpha_policy -- name of alpha changing policy function
        eps -- epsilon
        print_every -- print results every print_every steps
    """
    n=len(x_0)
    lower_bound = float("-inf")
    upper_bound = float("inf")
    
    
    criterion=1e10*eps
    x = x_0
    x_hist = []
    alpha_hist = []
    Gap_hist = []
    Q_hist = []
    time_hist = []
    grad_hist = []
    time_hist.append(0)
    int_start= time.time()
    print('********* Algorithm starts *********')
    max_iter=FW_params['iter_FW']
    line_search_tol=FW_params['line_search_tol']
    for k in range(1, max_iter + 1):
        start_time = time.time()
        Q, extra_params = fun_x(x)
        if min(extra_params)<-1e-10: #this is a way to know if the gradient is defined on x
            print("gradiend is not defined")
            break
            
        #find optimal
        grad = grad_x(x,extra_params)
        s=linear_oracle(grad)
       
        deltax=x-s  
        Gap = grad @ deltax
        lower_bound=max(lower_bound,Q-Gap)
        upper_bound=min(upper_bound,Q)
        
        if alpha_policy == 'standard':
            alpha = alpha_standard(k)
        elif alpha_policy == 'line_search':
            extra_param_s = extra_fun(s) #this is a way to know if the gradient is defined on s
            if min(extra_param_s)==0: #if 0 it is not defines and beta is adjusted
                beta=0.5
            else:
                beta=1  
            my_grad_beta= lambda beta:grad_beta(x,s,beta,extra_params,extra_param_s)   
            alpha = alpha_line_search(k,my_grad_beta,-deltax,beta,line_search_tol)
        elif alpha_policy == 'icml':
            hess_mult = hess_mult_x(s-x, extra_params)
            alpha = alpha_icml(Gap, hess_mult,-deltax,Mf,nu)
                      
        # filling history
        x_hist.append(x)
        alpha_hist.append(alpha)
        Gap_hist.append(Gap)
        Q_hist.append(Q)
        grad_hist.append(grad)
        time_hist.append(time.time() - start_time)
        
        x = x + alpha * (s - x)
        
        criterion = min(criterion,norm(x - x_hist[-1]) / max(1, norm(x_hist[-1])))
        #criterion = Gap
        #print(upper_bound)
        #print(lower_bound)
        #criterion=(upper_bound-lower_bound)/abs(lower_bound)
        if criterion <= eps:
            
            x_hist.append(x)
            Q_hist.append(Q)
            Q, _ = fun_x(x)
            print('Convergence achieved!')
            #print(f'x = {x}')
            #print(f'v = {v}')
            print(f'iter = {k}, stepsize = {alpha}, crit = {criterion}, upper_bound={upper_bound}, lower_bound={lower_bound}')
            return x, alpha_hist, Gap_hist, Q_hist, time_hist, grad_hist
                  
        
        if k % print_every == 0 or k == 1:
            if not debug_info:
                print(f'iter = {k}, stepsize = {alpha}, criterion = {criterion}, upper_bound={upper_bound}, lower_bound={lower_bound}')
            else:
                print(k)
                print(f'Q = {Q}')
                print(f's = {np.nonzero(s)}')
                print(f'Gap = {Gap}')
                print(f'alpha = {alpha}')
                print(f'criterion = {criterion}')
                #print(f'grad = {grad}')
                print(f'grad norm = {norm(grad)}')
                print(f'min abs dot = {min(abs(dot_product))}')
                #print(f'x = {x}')
                #x_nz = x[np.nonzero(x)[0]]
                #print(f'x non zero: {list(zip(x_nz, np.nonzero(x)[0]))}\n')
   
    x_hist.append(x)
    Q_hist.append(Q)
    int_end= time.time()
    print(int_end - int_start)
    return x, alpha_hist, Gap_hist, Q_hist, time_hist, grad_hist